In [43]:
from langchain_core.prompts import (SystemMessagePromptTemplate, 
                                    HumanMessagePromptTemplate,
                                    ChatPromptTemplate,
                                    PromptTemplate)

from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from pydantic import BaseModel, Field

In [4]:
system_prompt = SystemMessagePromptTemplate.from_template(
    "You are an helpful AI assistant that can summarize information from a text."
)

In [33]:
user_prompt = HumanMessagePromptTemplate.from_template(
    "You are able to summarize information contained in a text: {input}. Limit response to 100 charcters",
    input_variables=["input"]
)

In [8]:
test_string = """When buying farmland in India, you must check land title deeds and encumbrance certificates to ensure clear ownership and no financial liabilities, along with revenue records like the Record of Rights (ROR)/, mutation register, and property tax receipts to confirm tax compliance and a clean transaction history. Physical checks of the land, including boundary demarcation, soil quality, and water availability, are crucial. Additionally, be aware of state-specific rules, potential land ceiling limits, the need for conversion permissions, and zoning regulations. 
Legal & Documentation Checks
Title Deed & Ownership: Verify the seller's ownership and examine the title deed to ensure it's free from disputes. 
Encumbrance Certificate (EC): This certificate confirms the land is free from any financial or legal liabilities like mortgages. 
Record of Rights (ROR) / 7-12 Extract: This document provides a history of land ownership and cultivation, verifying past transactions. 
Mutation Register: Check for entries that confirm the land's ownership transfer over time. 
Property Tax & Land Tax Receipts: Ensure all land taxes are paid up to date. 
Survey Documents: Obtain documents like a survey map or sketch to confirm land boundaries. 
Legal Expertise: Consult a lawyer or real estate expert to review all documents for legal compliance and clarity. 
Physical & Land-Related Checks
Site Inspection: Visit the property to assess soil quality, water sources, and overall land condition. 
Water Availability: Check for sufficient water sources, such as groundwater or irrigation facilities, which impact land productivity. 
Road Connectivity: Evaluate access to the farmland via roads, as this affects its value and accessibility. 
Boundary Demarcation: Ensure the land's boundaries are clearly marked to prevent future disputes. 
Regulatory & State-Specific Checks
Land Classification & Use: Confirm the land's classification (e.g., for crop farming, horticulture) and check for zoning regulations that might restrict usage. 
State-Specific Restrictions: Research and understand the state's particular rules on who can buy agricultural land. 
Land Ceiling Limits: Be aware of state-specific limits on the maximum amount of agricultural land an individual can own. 
Conversion Permissions: If you plan to use the land for non-agricultural purposes, obtain the necessary conversion permissions from authorities. 
"""

In [13]:
user_prompt.format(input=test_string).content[:100]

'You are able to summarize information contained in a text: When buying farmland in India, you must c'

In [34]:
prompt = ChatPromptTemplate.from_messages([system_prompt, user_prompt])
[(message.__class__, message.input_variables) for message in prompt.messages]

[(langchain_core.prompts.chat.SystemMessagePromptTemplate, []),
 (langchain_core.prompts.chat.HumanMessagePromptTemplate, ['input'])]

In [16]:
llm = ChatOpenAI(model='gpt-4o-mini')

In [41]:
chain = (
    {"input": lambda x: x["input"]}
    | prompt
    | llm
    | StrOutputParser()
)

response = chain.invoke({"input": test_string})
response

'Check deeds, certificates, taxes; inspect land, verify regulations before buying farmland in India.'

In [42]:
system_prompt2 = SystemMessagePromptTemplate.from_template(
    "You are an expert on generating article titles."
)

user_prompt2 = HumanMessagePromptTemplate.from_template(
    "You are given an article: {article} and its summary: {article_summary}. Given these inputs, generate a suitable title for the article",
    input_variables=["article", "article_summary"]
)

promp2 = ChatPromptTemplate.from_messages([system_prompt2, user_prompt2])
chain2 = (
    {"article": lambda x: x["article"],
     "article_summary": lambda x: x["article_summary"]}
     | promp2
     | llm
     | StrOutputParser()
)

chain2.invoke({'article': test_string, 'article_summary': response})

'"Essential Guide to Buying Farmland in India: Navigating Legal Checks, Land Inspection, and Regulatory Compliance"'

In [44]:
class Article(BaseModel):
    original_text: str = Field(description="Original Text")
    title:str = Field(description="AI generated title")

In [49]:
structured_llm = llm.with_structured_output(Article)

chain2 = (
    {"article": lambda x: x["article"],
     "article_summary": lambda x: x["article_summary"]}
     | promp2
     | structured_llm
     | {"original_text": lambda x: x.original_text,
        "title": lambda x: x.title}
)

response = chain2.invoke({'article': test_string, 'article_summary': response})

In [51]:
print("Original text: " + response["original_text"][:100])
print("AI Generated title: " + response["title"])

Original text: When buying farmland in India, you must check land title deeds and encumbrance certificates to ensur
AI Generated title: Essential Checklist for Buying Farmland in India: Legal, Physical, and Regulatory Insights
